Build an ElectronicStructureProblem

In [4]:
'''PySCFDriver is a class that allows to build a molecluar object in PySCF. We use it to compute the one body and two body integrals.'''
from qiskit_nature.units import DistanceUnit
from qiskit_nature.second_q.drivers import PySCFDriver

driver = PySCFDriver(
    atom="H 0 0 0; H 0 0 0.735",
    basis="sto3g", #slater-type orbitals approximated with 3 gaussians
    charge=0, #int chrge of the molecule
    spin=0, #2S where S spin of the molecule
    unit=DistanceUnit.ANGSTROM,
)

In [5]:
# Problem is a ElectronicStructureProblem object
problem = driver.run()
print(problem)

In [6]:
# The ElectonicStructureProblem object contains the hamiltonian of the molecule, and electronic integrals which the one and two body integrals
hamiltonian = problem.hamiltonian
coefficients = hamiltonian.electronic_integrals #one and two body integrals are 2d and 4d tensors respectively
print(coefficients.alpha)

Polynomial Tensor
 "+-":
[[-1.25633907e+00 -1.37083854e-17]
 [-6.07732712e-17 -4.71896007e-01]]
 "++--":
[[[[6.75710155e-01 1.69253442e-16]
   [1.56722377e-16 1.80931200e-01]]

  [[4.84650299e-17 1.80931200e-01]
   [6.64581730e-01 3.79897400e-16]]]


 [[[1.01440795e-16 6.64581730e-01]
   [1.80931200e-01 4.71502663e-17]]

  [[1.80931200e-01 3.78920172e-16]
   [6.59828421e-17 6.98573723e-01]]]]


In [12]:
#The hamiltonian object contains the second quantized operator (the two summands of the hamiltonian). 
#This is the electronic part of the hamiltonian (nuclear repulsion energy will be added later as a constant, is available as hamiltonian.nuclear_repulsion_energy)
fermionic_op = hamiltonian.second_q_op()
print(fermionic_op)

Fermionic Operator
number spin orbitals=4, number terms=36
  -1.25633907300325 * ( +_0 -_0 )
+ -0.47189600728114184 * ( +_1 -_1 )
+ -1.25633907300325 * ( +_2 -_2 )
+ -0.47189600728114184 * ( +_3 -_3 )
+ 0.33785507740175813 * ( +_0 +_0 -_0 -_0 )
+ 0.09046559989211568 * ( +_0 +_0 -_1 -_1 )
+ 0.09046559989211564 * ( +_0 +_1 -_0 -_1 )
+ 0.33229086512764827 * ( +_0 +_1 -_1 -_0 )
+ 0.33785507740175813 * ( +_0 +_2 -_2 -_0 )
+ 0.09046559989211568 * ( +_0 +_2 -_3 -_1 )
+ 0.09046559989211564 * ( +_0 +_3 -_2 -_1 )
+ 0.33229086512764827 * ( +_0 +_3 -_3 -_0 )
+ 0.3322908651276482 * ( +_1 +_0 -_0 -_1 )
+ 0.09046559989211574 * ( +_1 +_0 -_1 -_0 )
+ 0.09046559989211565 * ( +_1 +_1 -_0 -_0 )
+ 0.3492868613660089 * ( +_1 +_1 -_1 -_1 )
+ 0.3322908651276482 * ( +_1 +_2 -_2 -_1 )
+ 0.09046559989211574 * ( +_1 +_2 -_3 -_0 )
+ 0.09046559989211565 * ( +_1 +_3 -_2 -_0 )
+ 0.3492868613660089 * ( +_1 +_3 -_3 -_1 )
+ 0.33785507740175813 * ( +_2 +_0 -_0 -_2 )
+ 0.09046559989211568 * ( +_2 +_0 -_1 -_3 )
+ 0.0904655

Mapping the problem to qubit space

In [8]:
'''Jordan-Wigner transformation maps the fermionic operators to unitary operators acting on qubits (https://arxiv.org/abs/2110.12792).
Doing so, we can map the n-body fermionic operator to a circuit of n-qubit gates.
'''
from qiskit_nature.mappers.second_quantization import JordanWignerMapper
mapper = JordanWignerMapper()

In [14]:
#Construct the qubit operator
#error 'FermionicOp' object has no attribute 'to_list' ????
qubit_op = mapper.map(fermionic_op)

AttributeError: 'FermionicOp' object has no attribute 'to_list'